In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly
import plotly.graph_objs as go
from tqdm import tqdm

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# folder = '/content/drive/My Drive/citadel datathon'
folder = '/content/drive/My Drive/Citadel APAC 2021/data'

player_df = pd.read_csv(folder + '/player.csv')
player_attr_df = pd.read_csv(folder + '/player_attributes.csv')
team_df = pd.read_csv(folder + '/team.csv', engine='python')
team_attr_df = pd.read_csv(folder + '/team_attributes.csv')

In [ ]:
new_player_df = pd.merge(player_df, player_attr_df, on='player_id').dropna(axis=0)

In [ ]:
new_player_df['preferred_foot'] = new_player_df['preferred_foot'].apply(lambda x: 1 if x == 'left' else 0)

In [ ]:
new_player_df['birthday'] = pd.to_datetime(new_player_df.birthday).dt.year

In [ ]:
new_player_df['date'] = pd.to_datetime(new_player_df.date)
new_player_df.drop(columns=['attacking_work_rate', 'defensive_work_rate'], inplace=True)

In [ ]:
new_player_df.to_csv(folder + '/new_player.csv')

In [ ]:
new_player_df

,player_id,player_name,birthday,height,weight,date,overall_rating,potential,preferred_foot,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,curve,free_kick_accuracy,long_passing,ball_control,acceleration,sprint_speed,agility,reactions,balance,shot_power,jumping,stamina,strength,long_shots,aggression,interceptions,positioning,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,505942,Aaron Appindangoye,1992,182.88,187,2016-02-18,67.0,71.0,0,49.0,44.0,71.0,61.0,44.0,51.0,45.0,39.0,64.0,49.0,60.0,64.0,59.0,47.0,65.0,55.0,58.0,54.0,76.0,35.0,71.0,70.0,45.0,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
1,505942,Aaron Appindangoye,1992,182.88,187,2015-11-19,67.0,71.0,0,49.0,44.0,71.0,61.0,44.0,51.0,45.0,39.0,64.0,49.0,60.0,64.0,59.0,47.0,65.0,55.0,58.0,54.0,76.0,35.0,71.0,70.0,45.0,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
2,505942,Aaron Appindangoye,1992,182.88,187,2015-09-21,62.0,66.0,0,49.0,44.0,71.0,61.0,44.0,51.0,45.0,39.0,64.0,49.0,60.0,64.0,59.0,47.0,65.0,55.0,58.0,54.0,76.0,35.0,63.0,41.0,45.0,54.0,48.0,65.0,66.0,69.0,6.0,11.0,10.0,8.0,8.0
3,505942,Aaron Appindangoye,1992,182.88,187,2015-03-20,61.0,65.0,0,48.0,43.0,70.0,60.0,43.0,50.0,44.0,38.0,63.0,48.0,60.0,64.0,59.0,46.0,65.0,54.0,58.0,54.0,76.0,34.0,62.0,40.0,44.0,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
4,505942,Aaron Appindangoye,1992,182.88,187,2007-02-22,61.0,65.0,0,48.0,43.0,70.0,60.0,43.0,50.0,44.0,38.0,63.0,48.0,60.0,64.0,59.0,46.0,65.0,54.0,58.0,54.0,76.0,34.0,62.0,40.0,44.0,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183973,39902,Zvjezdan Misimovic,1982,180.34,176,2009-08-30,83.0,85.0,0,84.0,77.0,59.0,89.0,77.0,84.0,86.0,78.0,84.0,85.0,66.0,72.0,77.0,86.0,73.0,76.0,58.0,72.0,67.0,81.0,56.0,78.0,86.0,88.0,83.0,22.0,31.0,30.0,9.0,20.0,84.0,20.0,20.0
183974,39902,Zvjezdan Misimovic,1982,180.34,176,2009-02-22,78.0,80.0,0,74.0,76.0,53.0,84.0,77.0,85.0,86.0,74.0,73.0,86.0,66.0,67.0,77.0,74.0,73.0,75.0,58.0,66.0,65.0,73.0,61.0,64.0,72.0,88.0,70.0,32.0,31.0,30.0,9.0,20.0,73.0,20.0,20.0
183975,39902,Zvjezdan Misimovic,1982,180.34,176,2008-08-30,77.0,80.0,0,74.0,71.0,53.0,84.0,77.0,85.0,86.0,74.0,73.0,86.0,66.0,67.0,77.0,74.0,73.0,75.0,58.0,66.0,65.0,73.0,67.0,64.0,72.0,88.0,70.0,32.0,31.0,30.0,9.0,20.0,73.0,20.0,20.0
183976,39902,Zvjezdan Misimovic,1982,180.34,176,2007-08-30,78.0,81.0,0,74.0,64.0,57.0,86.0,77.0,87.0,86.0,73.0,73.0,91.0,61.0,60.0,77.0,69.0,73.0,72.0,58.0,67.0,59.0,78.0,63.0,63.0,68.0,88.0,53.0,28.0,32.0,30.0,9.0,20.0,73.0,20.0,20.0


In [ ]:
new_player_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 180354 entries, 0 to 183977
Data columns (total 42 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   player_id           180354 non-null  int64         
 1   player_name         180354 non-null  object        
 2   birthday            180354 non-null  int64         
 3   height              180354 non-null  float64       
 4   weight              180354 non-null  int64         
 5   date                180354 non-null  datetime64[ns]
 6   overall_rating      180354 non-null  float64       
 7   potential           180354 non-null  float64       
 8   preferred_foot      180354 non-null  int64         
 9   crossing            180354 non-null  float64       
 10  finishing           180354 non-null  float64       
 11  heading_accuracy    180354 non-null  float64       
 12  short_passing       180354 non-null  float64       
 13  volleys             180354 no

In [3]:
match_data = pd.read_csv(folder + '/match.csv')

In [4]:
match_data = match_data.iloc[:, :32]  # delete bet data

In [5]:
match_data.drop(columns='season', inplace=True)

In [6]:
match_data.dropna(inplace=True)

In [7]:
match_data.iloc[:, 9:] = match_data.iloc[:, 9:].astype(int)

In [8]:
match_data.date = pd.to_datetime(match_data.date)

In [9]:
match_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21374 entries, 145 to 25978
Data columns (total 31 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   country_id      21374 non-null  int64         
 1   league_id       21374 non-null  int64         
 2   stage           21374 non-null  int64         
 3   date            21374 non-null  datetime64[ns]
 4   match_id        21374 non-null  int64         
 5   home_team_id    21374 non-null  int64         
 6   away_team_id    21374 non-null  int64         
 7   home_team_goal  21374 non-null  int64         
 8   away_team_goal  21374 non-null  int64         
 9   home_player_1   21374 non-null  int64         
 10  home_player_2   21374 non-null  int64         
 11  home_player_3   21374 non-null  int64         
 12  home_player_4   21374 non-null  int64         
 13  home_player_5   21374 non-null  int64         
 14  home_player_6   21374 non-null  int64         
 15  

In [10]:
match_data

,country_id,league_id,stage,date,match_id,home_team_id,away_team_id,home_team_goal,away_team_goal,home_player_1,home_player_2,home_player_3,home_player_4,home_player_5,home_player_6,home_player_7,home_player_8,home_player_9,home_player_10,home_player_11,away_player_1,away_player_2,away_player_3,away_player_4,away_player_5,away_player_6,away_player_7,away_player_8,away_player_9,away_player_10,away_player_11
145,1,1,24,2009-02-27,493017,8203,9987,2,1,38327,67950,67958,67959,37112,36393,148286,67898,164352,38801,26502,37937,38293,148313,104411,148314,37202,43158,9307,42153,32690,38782
153,1,1,25,2009-03-08,493025,9984,8342,1,3,36835,37047,37021,37051,104386,32863,37957,37909,38357,37065,78462,37990,21812,11736,37858,38366,37983,39578,38336,52280,27423,38440
155,1,1,25,2009-03-07,493027,8635,10000,2,0,34480,38388,26458,13423,38389,30949,38393,38253,38383,38778,37069,37900,37886,37903,37889,94030,37893,37981,131531,130027,38231,131530
162,1,1,26,2009-03-13,493034,8203,8635,2,1,38327,67950,67958,38801,67898,37112,67959,148286,164352,33657,26502,34480,38388,38389,31316,164694,30949,38378,38383,38393,38253,37069
168,1,1,26,2009-03-14,493040,10000,9999,0,0,37900,37886,37100,37903,37889,37893,37981,131531,131530,38231,130027,38318,38247,16387,94288,94284,45832,26669,33671,163670,37945,33622
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25972,24558,24558,8,2015-09-13,1992089,10243,10191,3,3,274776,451982,34268,197757,121080,462608,67008,632356,198082,194532,42262,67311,8800,320183,67304,67389,133126,186524,93223,121115,25809,289732
25974,24558,24558,9,2015-09-22,1992091,10190,10191,1,0,42231,678384,95220,638592,413155,45780,171229,67333,119839,143790,195215,462944,563066,8800,67304,158253,133126,186524,93223,121115,232110,289732
25975,24558,24558,9,2015-09-23,1992092,9824,10199,1,2,33272,41621,25813,257845,114735,42237,113227,358156,32343,531309,37257,42276,114792,150007,178119,27232,570830,260708,201704,36382,34082,95257
25976,24558,24558,9,2015-09-23,1992093,9956,10179,2,0,157856,274779,177689,294256,42258,39979,173936,147959,451983,80564,289472,10637,67349,202663,32597,114794,188114,25840,482200,95230,451335,275122


In [ ]:
def transform(col_add, df):
  temp = [col_add + '_' + i for i in df.columns]
  temp[0] = col_add
  temp[5] = 'date'
  df.columns = temp
  return df

In [ ]:
match_data = match_data.sort_values('date')
new_player_df = new_player_df.sort_values('date')
for i in match_data.columns[9:]:
  # df = transform(i, new_player_df.copy())
  # print(df.info())
  match_data = pd.merge_asof(match_data, transform(i, new_player_df.copy()), on='date', by=i)


In [ ]:
for col in match_data.columns:
  if 'birthday' in col:
    new_col = col.replace('birthday', 'age')
    match_data[new_col] =  match_data.date.dt.year - match_data[col]

In [ ]:
match_data

,country_id,league_id,stage,date,match_id,home_team_id,away_team_id,home_team_goal,away_team_goal,home_player_1,home_player_2,home_player_3,home_player_4,home_player_5,home_player_6,home_player_7,home_player_8,home_player_9,home_player_10,home_player_11,away_player_1,away_player_2,away_player_3,away_player_4,away_player_5,away_player_6,away_player_7,away_player_8,away_player_9,away_player_10,away_player_11,home_player_1_player_name,home_player_1_birthday,home_player_1_height,home_player_1_weight,home_player_1_overall_rating,home_player_1_potential,home_player_1_preferred_foot,home_player_1_crossing,home_player_1_finishing,...,away_player_11_shot_power,away_player_11_jumping,away_player_11_stamina,away_player_11_strength,away_player_11_long_shots,away_player_11_aggression,away_player_11_interceptions,away_player_11_positioning,away_player_11_vision,away_player_11_penalties,away_player_11_marking,away_player_11_standing_tackle,away_player_11_sliding_tackle,away_player_11_gk_diving,away_player_11_gk_handling,away_player_11_gk_kicking,away_player_11_gk_positioning,away_player_11_gk_reflexes,home_player_1_age,home_player_2_age,home_player_3_age,home_player_4_age,home_player_5_age,home_player_6_age,home_player_7_age,home_player_8_age,home_player_9_age,home_player_10_age,home_player_11_age,away_player_1_age,away_player_2_age,away_player_3_age,away_player_4_age,away_player_5_age,away_player_6_age,away_player_7_age,away_player_8_age,away_player_9_age,away_player_10_age,away_player_11_age
0,4769,4769,1,2008-08-09,483130,9827,7819,2,1,30458,32571,38703,40543,26119,39978,39962,33744,39963,31292,25537,11321,11324,26412,46338,11327,26420,26191,11329,18913,35410,46261,Ulrich Rame,1972.0,187.96,187.0,81.0,87.0,0.0,21.0,21.0,...,46.0,58.0,50.0,49.0,72.0,46.0,50.0,47.0,53.0,57.0,22.0,25.0,22.0,6.0,22.0,38.0,22.0,22.0,36.0,NaN,30.0,28.0,26.0,27.0,27.0,22.0,26.0,22.0,24.0,28.0,29.0,NaN,25.0,29.0,31.0,30.0,25.0,28.0,NaN,23.0
1,4769,4769,1,2008-08-09,483134,9829,9847,1,0,145039,41097,26216,39199,38729,71603,26120,32566,33435,51371,40762,30742,20482,94043,26357,30457,30695,35499,32578,32572,37554,26312,Jean-Jacques Gosso,1983.0,177.80,165.0,69.0,75.0,0.0,61.0,30.0,...,69.0,72.0,69.0,71.0,60.0,55.0,33.0,65.0,57.0,71.0,15.0,21.0,11.0,6.0,14.0,12.0,14.0,6.0,25.0,22.0,30.0,26.0,30.0,19.0,26.0,29.0,28.0,21.0,20.0,29.0,28.0,18.0,29.0,28.0,35.0,24.0,30.0,32.0,24.0,28.0
2,4769,4769,1,2008-08-09,483131,9746,9831,1,0,41186,71607,38111,46447,103470,46651,46647,109590,46656,26405,41146,33617,26356,41121,26304,41019,30908,41122,26325,39987,94318,38108,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,73.0,66.0,71.0,69.0,65.0,63.0,38.0,75.0,58.0,70.0,28.0,36.0,28.0,12.0,20.0,24.0,20.0,20.0,NaN,22.0,31.0,NaN,19.0,24.0,27.0,NaN,31.0,29.0,29.0,NaN,NaN,29.0,NaN,29.0,24.0,23.0,33.0,21.0,20.0,26.0
3,4769,4769,1,2008-08-09,483135,8481,8639,0,0,26233,111909,31684,94313,26245,25538,46757,26227,26238,38951,26494,26150,26107,26149,77741,26154,26291,26160,20401,32575,35407,26156,Gennaro Bracigliano,1980.0,182.88,181.0,76.0,79.0,0.0,22.0,22.0,...,71.0,80.0,53.0,61.0,64.0,56.0,52.0,42.0,60.0,49.0,37.0,42.0,53.0,11.0,20.0,54.0,20.0,20.0,28.0,24.0,NaN,21.0,28.0,28.0,26.0,28.0,23.0,21.0,27.0,35.0,25.0,32.0,23.0,23.0,28.0,22.0,25.0,24.0,28.0,24.0
4,4769,4769,1,2008-08-09,483137,9874,9855,1,2,26359,147351,26345,26328,41178,36396,26371,26370,26374,34983,41238,26190,41302,26027,32755,46709,31037,93480,5217,26285,33599,103482,Teddy Richert,1974.0,182.88,174.0,78.0,84.0,0.0,22.0,22.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34.0,22.0,24.0,24.0,NaN,29.0,31.0,33.0,21.0,28.0,23.0,NaN,NaN,31.0,NaN,25.0,24.0,19.0,33.0,24.0,31.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21369,24558,

In [ ]:
# match_data.to_csv('match_player.csv', index=False)

In [ ]:
match_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21374 entries, 0 to 21373
Columns: 933 entries, country_id to away_player_11_age
dtypes: datetime64[ns](1), float64(880), int64(30), object(22)
memory usage: 152.3+ MB


In [ ]:
team_attr_df['date'] = pd.to_datetime(team_attr_df['date'])
del team_attr_df['buildUpPlayDribbling']
team_attr_df = pd.get_dummies(team_attr_df)
new_team_df = pd.merge(team_df, team_attr_df, on='team_id').dropna(axis=0)

In [12]:
team_attr_df['date'] = pd.to_datetime(team_attr_df['date'])
team_attr_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1458 entries, 0 to 1457
Data columns (total 23 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   team_id                         1458 non-null   int64         
 1   date                            1458 non-null   datetime64[ns]
 2   buildUpPlaySpeed                1458 non-null   int64         
 3   buildUpPlaySpeedClass           1458 non-null   object        
 4   buildUpPlayDribbling            489 non-null    float64       
 5   buildUpPlayDribblingClass       1458 non-null   object        
 6   buildUpPlayPassing              1458 non-null   int64         
 7   buildUpPlayPassingClass         1458 non-null   object        
 8   buildUpPlayPositioningClass     1458 non-null   object        
 9   chanceCreationPassing           1458 non-null   int64         
 10  chanceCreationPassingClass      1458 non-null   object        
 11  chan

In [21]:
for column in team_attr_df.columns:
  if team_attr_df[column].dtype == np.object:
    # print(column)
    del team_attr_df[column]

team_attr_df

,team_id,date,buildUpPlaySpeed,buildUpPlayDribbling,buildUpPlayPassing,chanceCreationPassing,chanceCreationCrossing,chanceCreationShooting,defencePressure,defenceAggression,defenceTeamWidth
0,9930,2010-02-22,60,NaN,50,60,65,55,50,55,45
1,9930,2014-09-19,52,48.0,56,54,63,64,47,44,54
2,9930,2015-09-10,47,41.0,54,54,63,64,47,44,54
3,8485,2010-02-22,70,NaN,70,70,70,70,60,70,70
4,8485,2011-02-22,47,NaN,52,53,48,52,47,47,52
...,...,...,...,...,...,...,...,...,...,...,...
1453,10000,2011-02-22,52,NaN,52,52,48,53,46,48,53
1454,10000,2012-02-22,54,NaN,51,47,52,50,44,55,53
1455,10000,2013-09-20,54,NaN,51,47,52,32,44,58,37
1456,10000,2014-09-19,54,42.0,51,47,52,32,44,58,37


In [ ]:
new_team_df

,team_id,team_long_name,team_short_name,date,buildUpPlaySpeed,buildUpPlayPassing,chanceCreationPassing,chanceCreationCrossing,chanceCreationShooting,defencePressure,defenceAggression,defenceTeamWidth,buildUpPlaySpeedClass_Balanced,buildUpPlaySpeedClass_Fast,buildUpPlaySpeedClass_Slow,buildUpPlayDribblingClass_Little,buildUpPlayDribblingClass_Lots,buildUpPlayDribblingClass_Normal,buildUpPlayPassingClass_Long,buildUpPlayPassingClass_Mixed,buildUpPlayPassingClass_Short,buildUpPlayPositioningClass_Free Form,buildUpPlayPositioningClass_Organised,chanceCreationPassingClass_Normal,chanceCreationPassingClass_Risky,chanceCreationPassingClass_Safe,chanceCreationCrossingClass_Little,chanceCreationCrossingClass_Lots,chanceCreationCrossingClass_Normal,chanceCreationShootingClass_Little,chanceCreationShootingClass_Lots,chanceCreationShootingClass_Normal,chanceCreationPositioningClass_Free Form,chanceCreationPositioningClass_Organised,defencePressureClass_Deep,defencePressureClass_High,defencePressureClass_Medium,defenceAggressionClass_Contain,defenceAggressionClass_Double,defenceAggressionClass_Press,defenceTeamWidthClass_Narrow,defenceTeamWidthClass_Normal,defenceTeamWidthClass_Wide,defenceDefenderLineClass_Cover,defenceDefenderLineClass_Offside Trap
0,9987,KRC Genk,GEN,2010-02-22,45,45,50,35,60,70,65,70,1,0,0,1,0,0,0,1,0,0,1,1,0,0,0,0,1,0,0,1,0,1,0,1,0,0,0,1,0,0,1,1,0
1,9987,KRC Genk,GEN,2011-02-22,66,52,65,66,51,48,47,54,1,0,0,1,0,0,0,1,0,0,1,1,0,0,0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0,0,1
2,9987,KRC Genk,GEN,2012-02-22,53,55,55,48,56,47,45,55,1,0,0,1,0,0,0,1,0,0,1,1,0,0,0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0,1,0
3,9987,KRC Genk,GEN,2013-09-20,58,38,67,48,56,47,45,55,1,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0,1,0
4,9987,KRC Genk,GEN,2014-09-19,58,38,67,48,56,47,45,55,1,0,0,0,0,1,0,1,0,0,1,0,1,0,0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1453,9777,Servette FC,SER,2013-09-20,48,51,53,66,45,49,45,49,1,0,0,1,0,0,0,1,0,0,1,1,0,0,0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0,1,0
1454,7730,FC Lausanne-Sports,LAU,2010-02-22,30,60,40,60,60,55,60,50,0,0,1,1,0,0,0,1,0,0,1,1,0,0,0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0,1,0
1455,7730,FC Lausanne-Sports,LAU,2012-02-22,37,49,52,60,48,43,43,55,1,0,0,1,0,0,0,1,0,0,1,1,0,0,0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0,1,0
1456,7730,FC Lausanne-Sports,LAU,2013-09-20,51,49,52,60,48,43,43,55,1,0,0,1,0,0,0,1,0,0,1,1,0,0,0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0,1,0


In [25]:
def transform(col_add, df):
  temp = [col_add + '_' + i for i in df.columns]
  temp[0] = col_add
  temp[1] = 'date'
  df.columns = temp
  return df

In [27]:
match_data = match_data.sort_values('date')
team_attr_df = team_attr_df.sort_values('date')
for i in ['home_team_id', 'away_team_id']:
  # df = transform(i, new_team_df.copy())
  # print(df.columns)
  match_data = pd.merge_asof(match_data, transform(i, team_attr_df.copy()), on='date', by=i)

In [28]:
match_data

,country_id,league_id,stage,date,match_id,home_team_id,away_team_id,home_team_goal,away_team_goal,home_player_1,home_player_2,home_player_3,home_player_4,home_player_5,home_player_6,home_player_7,home_player_8,home_player_9,home_player_10,home_player_11,away_player_1,away_player_2,away_player_3,away_player_4,away_player_5,away_player_6,away_player_7,away_player_8,away_player_9,away_player_10,away_player_11,home_team_id_buildUpPlaySpeed,home_team_id_buildUpPlayDribbling,home_team_id_buildUpPlayPassing,home_team_id_chanceCreationPassing,home_team_id_chanceCreationCrossing,home_team_id_chanceCreationShooting,home_team_id_defencePressure,home_team_id_defenceAggression,home_team_id_defenceTeamWidth,away_team_id_buildUpPlaySpeed,away_team_id_buildUpPlayDribbling,away_team_id_buildUpPlayPassing,away_team_id_chanceCreationPassing,away_team_id_chanceCreationCrossing,away_team_id_chanceCreationShooting,away_team_id_defencePressure,away_team_id_defenceAggression,away_team_id_defenceTeamWidth
0,4769,4769,1,2008-08-09,483130,9827,7819,2,1,30458,32571,38703,40543,26119,39978,39962,33744,39963,31292,25537,11321,11324,26412,46338,11327,26420,26191,11329,18913,35410,46261,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4769,4769,1,2008-08-09,483134,9829,9847,1,0,145039,41097,26216,39199,38729,71603,26120,32566,33435,51371,40762,30742,20482,94043,26357,30457,30695,35499,32578,32572,37554,26312,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4769,4769,1,2008-08-09,483131,9746,9831,1,0,41186,71607,38111,46447,103470,46651,46647,109590,46656,26405,41146,33617,26356,41121,26304,41019,30908,41122,26325,39987,94318,38108,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4769,4769,1,2008-08-09,483135,8481,8639,0,0,26233,111909,31684,94313,26245,25538,46757,26227,26238,38951,26494,26150,26107,26149,77741,26154,26291,26160,20401,32575,35407,26156,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4769,4769,1,2008-08-09,483137,9874,9855,1,2,26359,147351,26345,26328,41178,36396,26371,26370,26374,34983,41238,26190,41302,26027,32755,46709,31037,93480,5217,26285,33599,103482,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21369,24558,24558,36,2016-05-25,1992226,7896,10190,3,0,330458,282287,429986,41415,114212,340790,491221,8893,406283,614454,178142,25779,678384,638592,413155,210423,45780,176298,119839,566785,92094,35831,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,52.0,46.0,48.0,39.0,39.0,37.0,40.0,36.0,52.0
21370,24558,24558,36,2016-05-25,1992227,10199,10179,2,2,42276,114792,150007,178119,42294,32343,260708,92252,95257,36382,34082,441302,67349,450493,32597,140495,106559,25840,634310,95230,427439,38601,63.0,51.0,51.0,54.0,40.0,66.0,48.0,53.0,50.0,61.0,57.0,37.0,62.0,50.0,53.0,45.0,47.0,54.0
21371,24558,24558,36,2016-05-25,1992228,10191,10192,0,3,462944,239959,67304,320183,42257,93223,133126,112981,67389,232110,113237,25947,25843,108451,25815,733787,25860,245161,320184,45174,71764,37554,58.0,59.0,35.0,64.0,36.0,66.0,47.0,45.0,53.0,52.0,56.0,64.0,39.0,66.0,46.0,44.0,34.0,50.0
21372,24558,24558,36,2016-05-25,1992225,9931,9956,0,1,42113,458806,705484,22834,181211,438780,114738,25794,34035,40274,726956,132814,274779,177689,696435,42258,173936,39979,614540,291106,80564,289472,61.0,63.0,46.0,66.0,66.0,54.0,47.0,58.0,53.0,50.0,60.0,54.0,52.0,54.0,46.0,36.0,43.0,47.0


In [ ]:
match_data.to_csv(folder + '/match_total.csv', index=False)

## Team Quantative Importance

In [85]:
# folder = '/content/drive/My Drive/citadel datathon'
folder = '/content/drive/My Drive/Citadel APAC 2021/data'

player_df = pd.read_csv(folder + '/player.csv')
player_attr_df = pd.read_csv(folder + '/player_attributes.csv')
team_df = pd.read_csv(folder + '/team.csv', engine='python')
team_attr_df = pd.read_csv(folder + '/team_attributes.csv')

In [86]:
match_data = pd.read_csv(folder + '/match.csv')

In [87]:
match_data = match_data.iloc[:, :32]  # delete bet data

In [88]:
match_data.drop(columns='season', inplace=True)

In [89]:
match_data.dropna(inplace=True)

In [90]:
match_data.iloc[:, 9:] = match_data.iloc[:, 9:].astype(int)

In [91]:
match_data.date = pd.to_datetime(match_data.date)

In [92]:
match_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21374 entries, 145 to 25978
Data columns (total 31 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   country_id      21374 non-null  int64         
 1   league_id       21374 non-null  int64         
 2   stage           21374 non-null  int64         
 3   date            21374 non-null  datetime64[ns]
 4   match_id        21374 non-null  int64         
 5   home_team_id    21374 non-null  int64         
 6   away_team_id    21374 non-null  int64         
 7   home_team_goal  21374 non-null  int64         
 8   away_team_goal  21374 non-null  int64         
 9   home_player_1   21374 non-null  int64         
 10  home_player_2   21374 non-null  int64         
 11  home_player_3   21374 non-null  int64         
 12  home_player_4   21374 non-null  int64         
 13  home_player_5   21374 non-null  int64         
 14  home_player_6   21374 non-null  int64         
 15  

In [93]:
team_attr_df['date'] = pd.to_datetime(team_attr_df['date'])
team_attr_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1458 entries, 0 to 1457
Data columns (total 23 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   team_id                         1458 non-null   int64         
 1   date                            1458 non-null   datetime64[ns]
 2   buildUpPlaySpeed                1458 non-null   int64         
 3   buildUpPlaySpeedClass           1458 non-null   object        
 4   buildUpPlayDribbling            489 non-null    float64       
 5   buildUpPlayDribblingClass       1458 non-null   object        
 6   buildUpPlayPassing              1458 non-null   int64         
 7   buildUpPlayPassingClass         1458 non-null   object        
 8   buildUpPlayPositioningClass     1458 non-null   object        
 9   chanceCreationPassing           1458 non-null   int64         
 10  chanceCreationPassingClass      1458 non-null   object        
 11  chan

In [94]:
for column in team_attr_df.columns:
  if team_attr_df[column].dtype == np.object:
    # print(column)
    del team_attr_df[column]

team_attr_df

,team_id,date,buildUpPlaySpeed,buildUpPlayDribbling,buildUpPlayPassing,chanceCreationPassing,chanceCreationCrossing,chanceCreationShooting,defencePressure,defenceAggression,defenceTeamWidth
0,9930,2010-02-22,60,NaN,50,60,65,55,50,55,45
1,9930,2014-09-19,52,48.0,56,54,63,64,47,44,54
2,9930,2015-09-10,47,41.0,54,54,63,64,47,44,54
3,8485,2010-02-22,70,NaN,70,70,70,70,60,70,70
4,8485,2011-02-22,47,NaN,52,53,48,52,47,47,52
...,...,...,...,...,...,...,...,...,...,...,...
1453,10000,2011-02-22,52,NaN,52,52,48,53,46,48,53
1454,10000,2012-02-22,54,NaN,51,47,52,50,44,55,53
1455,10000,2013-09-20,54,NaN,51,47,52,32,44,58,37
1456,10000,2014-09-19,54,42.0,51,47,52,32,44,58,37


In [95]:
team_attr_df.columns

Index(['team_id', 'date', 'buildUpPlaySpeed', 'buildUpPlayDribbling',
       'buildUpPlayPassing', 'chanceCreationPassing', 'chanceCreationCrossing',
       'chanceCreationShooting', 'defencePressure', 'defenceAggression',
       'defenceTeamWidth'],
      dtype='object')

In [96]:
def transform(col_add, df):
  temp = [col_add + '_' + i for i in df.columns]
  temp[0] = col_add
  temp[1] = 'date'
  df.columns = temp
  return df

In [97]:
match_data = match_data.sort_values('date')
team_attr_df = team_attr_df.sort_values('date')
for i in ['home_team_id', 'away_team_id']:
  # df = transform(i, new_team_df.copy())
  # print(df.columns)
  match_data = pd.merge_asof(match_data, transform(i, team_attr_df.copy()), on='date', by=i)

In [98]:
match_data

,country_id,league_id,stage,date,match_id,home_team_id,away_team_id,home_team_goal,away_team_goal,home_player_1,home_player_2,home_player_3,home_player_4,home_player_5,home_player_6,home_player_7,home_player_8,home_player_9,home_player_10,home_player_11,away_player_1,away_player_2,away_player_3,away_player_4,away_player_5,away_player_6,away_player_7,away_player_8,away_player_9,away_player_10,away_player_11,home_team_id_buildUpPlaySpeed,home_team_id_buildUpPlayDribbling,home_team_id_buildUpPlayPassing,home_team_id_chanceCreationPassing,home_team_id_chanceCreationCrossing,home_team_id_chanceCreationShooting,home_team_id_defencePressure,home_team_id_defenceAggression,home_team_id_defenceTeamWidth,away_team_id_buildUpPlaySpeed,away_team_id_buildUpPlayDribbling,away_team_id_buildUpPlayPassing,away_team_id_chanceCreationPassing,away_team_id_chanceCreationCrossing,away_team_id_chanceCreationShooting,away_team_id_defencePressure,away_team_id_defenceAggression,away_team_id_defenceTeamWidth
0,4769,4769,1,2008-08-09,483130,9827,7819,2,1,30458,32571,38703,40543,26119,39978,39962,33744,39963,31292,25537,11321,11324,26412,46338,11327,26420,26191,11329,18913,35410,46261,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4769,4769,1,2008-08-09,483134,9829,9847,1,0,145039,41097,26216,39199,38729,71603,26120,32566,33435,51371,40762,30742,20482,94043,26357,30457,30695,35499,32578,32572,37554,26312,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4769,4769,1,2008-08-09,483131,9746,9831,1,0,41186,71607,38111,46447,103470,46651,46647,109590,46656,26405,41146,33617,26356,41121,26304,41019,30908,41122,26325,39987,94318,38108,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4769,4769,1,2008-08-09,483135,8481,8639,0,0,26233,111909,31684,94313,26245,25538,46757,26227,26238,38951,26494,26150,26107,26149,77741,26154,26291,26160,20401,32575,35407,26156,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4769,4769,1,2008-08-09,483137,9874,9855,1,2,26359,147351,26345,26328,41178,36396,26371,26370,26374,34983,41238,26190,41302,26027,32755,46709,31037,93480,5217,26285,33599,103482,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21369,24558,24558,36,2016-05-25,1992225,9931,9956,0,1,42113,458806,705484,22834,181211,438780,114738,25794,34035,40274,726956,132814,274779,177689,696435,42258,173936,39979,614540,291106,80564,289472,61.0,63.0,46.0,66.0,66.0,54.0,47.0,58.0,53.0,50.0,60.0,54.0,52.0,54.0,46.0,36.0,43.0,47.0
21370,24558,24558,36,2016-05-25,1992226,7896,10190,3,0,330458,282287,429986,41415,114212,340790,491221,8893,406283,614454,178142,25779,678384,638592,413155,210423,45780,176298,119839,566785,92094,35831,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,52.0,46.0,48.0,39.0,39.0,37.0,40.0,36.0,52.0
21371,24558,24558,36,2016-05-25,1992227,10199,10179,2,2,42276,114792,150007,178119,42294,32343,260708,92252,95257,36382,34082,441302,67349,450493,32597,140495,106559,25840,634310,95230,427439,38601,63.0,51.0,51.0,54.0,40.0,66.0,48.0,53.0,50.0,61.0,57.0,37.0,62.0,50.0,53.0,45.0,47.0,54.0
21372,24558,24558,36,2016-05-25,1992228,10191,10192,0,3,462944,239959,67304,320183,42257,93223,133126,112981,67389,232110,113237,25947,25843,108451,25815,733787,25860,245161,320184,45174,71764,37554,58.0,59.0,35.0,64.0,36.0,66.0,47.0,45.0,53.0,52.0,56.0,64.0,39.0,66.0,46.0,44.0,34.0,50.0


In [99]:
match_data.describe()

,country_id,league_id,stage,match_id,home_team_id,away_team_id,home_team_goal,away_team_goal,home_player_1,home_player_2,home_player_3,home_player_4,home_player_5,home_player_6,home_player_7,home_player_8,home_player_9,home_player_10,home_player_11,away_player_1,away_player_2,away_player_3,away_player_4,away_player_5,away_player_6,away_player_7,away_player_8,away_player_9,away_player_10,away_player_11,home_team_id_buildUpPlaySpeed,home_team_id_buildUpPlayDribbling,home_team_id_buildUpPlayPassing,home_team_id_chanceCreationPassing,home_team_id_chanceCreationCrossing,home_team_id_chanceCreationShooting,home_team_id_defencePressure,home_team_id_defenceAggression,home_team_id_defenceTeamWidth,away_team_id_buildUpPlaySpeed,away_team_id_buildUpPlayDribbling,away_team_id_buildUpPlayPassing,away_team_id_chanceCreationPassing,away_team_id_chanceCreationCrossing,away_team_id_chanceCreationShooting,away_team_id_defencePressure,away_team_id_defenceAggression,away_team_id_defenceTeamWidth
count,21374.000000,21374.000000,21374.000000,2.137400e+04,21374.000000,21374.000000,21374.000000,21374.000000,21374.000000,21374.000000,21374.000000,21374.000000,21374.000000,21374.000000,21374.000000,21374.000000,21374.000000,21374.000000,21374.000000,21374.000000,21374.000000,21374.000000,21374.000000,21374.000000,21374.000000,21374.000000,21374.000000,21374.000000,21374.000000,21374.000000,17528.000000,5388.000000,17528.000000,17528.000000,17528.000000,17528.000000,17528.000000,17528.000000,17528.000000,17545.000000,5384.000000,17545.000000,17545.000000,17545.000000,17545.000000,17545.000000,17545.000000,17545.000000
mean,11228.879199,11228.879199,18.580893,1.251568e+06,9977.827781,10048.081969,1.557266,1.175073,76742.444231,107817.580986,92216.855432,94926.233976,110463.847104,103180.635960,98180.729718,108240.828062,112384.670955,105535.011322,103474.254468,76917.826705,108366.129690,91374.264012,95044.389164,110431.136989,103257.580284,98863.244222,110122.267708,112000.940722,107360.801722,105299.890053,52.849498,48.933185,48.065895,52.553115,54.514320,53.904895,46.305853,49.458980,52.258101,52.818125,48.907318,48.094956,52.559020,54.491992,53.915645,46.310630,49.455514,52.259219
std,7639.283384,7639.283384,10.469817,4.805469e+05,13147.549402,13579.703256,1.303900,1.149778,87772.824162,113107.414135,103610.454488,103048.540630,113913.767568,112438.370759,108107.615413,115320.342312,117140.476897,111743.993319,109311.685174,88071.261260,114054.960365,102137.928138,103716.234711,113137.976484,111596.132480,108734.466569,117333.639465,116984.819126,113083.611508,111132.868797,11.632157,9.830416,11.059384,10.454668,11.024774,10.372160,10.025247,9.635742,9.314198,11.650688,9.850798,11.070922,10.474344,11.029189,10.375610,10.041918,9.612889,9.306937
min,1.000000,1.000000,1.000000,4.831300e+05,1601.000000,1601.000000,0.000000,0.000000,2984.000000,2802.000000,2752.000000,2752.000000,2752.000000,2770.000000,2625.000000,2802.000000,2770.000000,2770.000000,2802.000000,2796.000000,2790.000000,2752.000000,2752.000000,2790.000000,2802.000000,2625.000000,2770.000000,2770.000000,2770.000000,2802.000000,20.000000,24.000000,20.000000,21.000000,20.000000,22.000000,23.000000,24.000000,30.000000,20.000000,24.000000,20.000000,21.000000,20.000000,22.000000,23.000000,24.000000,30.000000
25%,4769.000000,4769.000000,10.000000,8.401552e+05,8530.000000,8530.000000,1.000000,0.000000,30622.000000,32553.000000,30459.000000,30617.000000,33559.500000,30994.000000,30889.000000,32745.000000,33348.000000,31962.000000,32465.000000,30622.000000,32579.000000,30453.000000,30593.000000,33338.000000,31011.000000,30907.000000,32761.000000,33435.250000,32343.000000,32674.000000,46.000000,42.000000,40.000000,47.000000,48.000000,48.000000,39.000000,44.000000,48.000000,46.000000,42.000000,40.000000,47.000000,48.000000,48.000000,39.000000,44.000000,48.000000
50%,10257.000000,10257.000000,19.000000,1.227826e+06,9747.000000,9747.000000,1.000000,1.000000,37900.000000,42258.000000,39643.000000,40735.

In [100]:
match_data.columns

Index(['country_id', 'league_id', 'stage', 'date', 'match_id', 'home_team_id',
       'away_team_id', 'home_team_goal', 'away_team_goal', 'home_player_1',
       'home_player_2', 'home_player_3', 'home_player_4', 'home_player_5',
       'home_player_6', 'home_player_7', 'home_player_8', 'home_player_9',
       'home_player_10', 'home_player_11', 'away_player_1', 'away_player_2',
       'away_player_3', 'away_player_4', 'away_player_5', 'away_player_6',
       'away_player_7', 'away_player_8', 'away_player_9', 'away_player_10',
       'away_player_11', 'home_team_id_buildUpPlaySpeed',
       'home_team_id_buildUpPlayDribbling', 'home_team_id_buildUpPlayPassing',
       'home_team_id_chanceCreationPassing',
       'home_team_id_chanceCreationCrossing',
       'home_team_id_chanceCreationShooting', 'home_team_id_defencePressure',
       'home_team_id_defenceAggression', 'home_team_id_defenceTeamWidth',
       'away_team_id_buildUpPlaySpeed', 'away_team_id_buildUpPlayDribbling',
       'aw

In [101]:
diff_attribute_list = ['buildUpPlaySpeed', 'buildUpPlayDribbling',
       'buildUpPlayPassing', 'chanceCreationPassing', 'chanceCreationCrossing',
       'chanceCreationShooting', 'defencePressure', 'defenceAggression',
       'defenceTeamWidth']
for attribute in diff_attribute_list:
  home_column = match_data['home_team_id_'+attribute]
  away_column = match_data['away_team_id_'+attribute]
  diff_column = match_data['home_team_id_'+attribute] - match_data['away_team_id_'+attribute]
  match_data['diff_'+attribute] = diff_column

In [102]:
for attribute in ['goal']:
  home_column = match_data['home_team_'+attribute]
  away_column = match_data['away_team_'+attribute]
  diff_column = match_data['home_team_'+attribute] - match_data['away_team_'+attribute]
  match_data['diff_'+attribute] = diff_column

In [103]:
match_data.to_csv(folder + '/match_quantative_total.csv', index=False)

In [104]:
match_data.columns

Index(['country_id', 'league_id', 'stage', 'date', 'match_id', 'home_team_id',
       'away_team_id', 'home_team_goal', 'away_team_goal', 'home_player_1',
       'home_player_2', 'home_player_3', 'home_player_4', 'home_player_5',
       'home_player_6', 'home_player_7', 'home_player_8', 'home_player_9',
       'home_player_10', 'home_player_11', 'away_player_1', 'away_player_2',
       'away_player_3', 'away_player_4', 'away_player_5', 'away_player_6',
       'away_player_7', 'away_player_8', 'away_player_9', 'away_player_10',
       'away_player_11', 'home_team_id_buildUpPlaySpeed',
       'home_team_id_buildUpPlayDribbling', 'home_team_id_buildUpPlayPassing',
       'home_team_id_chanceCreationPassing',
       'home_team_id_chanceCreationCrossing',
       'home_team_id_chanceCreationShooting', 'home_team_id_defencePressure',
       'home_team_id_defenceAggression', 'home_team_id_defenceTeamWidth',
       'away_team_id_buildUpPlaySpeed', 'away_team_id_buildUpPlayDribbling',
       'aw